In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tree_model").getOrCreate()
data = spark.read.csv("dog_food.csv", inferSchema=True ,header=True)
data.printSchema()
data.show(5)

22/09/13 20:36:34 WARN Utils: Your hostname, TIGER03366 resolves to a loopback address: 127.0.1.1; using 172.18.162.239 instead (on interface eth0)
22/09/13 20:36:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/13 20:36:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/13 20:36:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
+---+---+----+---+-------+
only showing top 5 rows



In [2]:
data.groupBy("Spoiled").count().show()

+-------+-----+
|Spoiled|count|
+-------+-----+
|    0.0|  350|
|    1.0|  140|
+-------+-----+



In [4]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [6]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["A","B","C","D"],outputCol="features")
output = assembler.transform(data)

In [7]:
output.show(5)

+---+---+----+---+-------+------------------+
|  A|  B|   C|  D|Spoiled|          features|
+---+---+----+---+-------+------------------+
|  4|  2|12.0|  3|    1.0|[4.0,2.0,12.0,3.0]|
|  5|  6|12.0|  7|    1.0|[5.0,6.0,12.0,7.0]|
|  6|  2|13.0|  6|    1.0|[6.0,2.0,13.0,6.0]|
|  4|  2|12.0|  1|    1.0|[4.0,2.0,12.0,1.0]|
|  4|  2|12.0|  3|    1.0|[4.0,2.0,12.0,3.0]|
+---+---+----+---+-------+------------------+
only showing top 5 rows



In [8]:
final_data = output.select(["features" , "Spoiled"])

In [9]:
from pyspark.ml.classification import (DecisionTreeClassifier , RandomForestClassifier , GBTClassifier)
dtc = DecisionTreeClassifier(labelCol="Spoiled" , featuresCol="features")
rfc = RandomForestClassifier(labelCol="Spoiled" , featuresCol="features")
gbt = GBTClassifier(labelCol="Spoiled" , featuresCol="features")

In [10]:
dtc_model = dtc.fit(final_data)
rfc_model = rfc.fit(final_data)
gbt_model = gbt.fit(final_data)

In [12]:
print(dtc_model.featureImportances)
print(rfc_model.featureImportances)
print(gbt_model.featureImportances)

(4,[1,2,3],[0.0019107795086908742,0.9831676511855764,0.014921569305732818])
(4,[0,1,2,3],[0.02333745785538593,0.024936812335310133,0.9273655883297189,0.024360141479585047])
(4,[0,1,2,3],[0.02962567485294246,0.03830179415146122,0.8286277188140007,0.10344481218159562])
